In [ ]:
!pip install tensorflow



In [ ]:
!pip install nltk['punkt']
# install gensim
!pip install gensim
# install spacy
!pip install spacy
!pip install plotly

In [ ]:
from collections import Counter
import operator
import plotly.express as px
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, TimeDistributed, RepeatVector, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
#from jupyterthemes import jtplot
#jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_english = pd.read_csv('/content/drive/MyDrive/ML files/small_vocab_en.csv', sep = '/t', names = ['english'])
df_french = pd.read_csv('/content/drive/MyDrive/ML files/small_vocab_fr.csv', sep = '/t', names = ['french'])

<ipython-input-142-a62a9b17b262>:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.

<ipython-input-142-a62a9b17b262>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [ ]:
df_english


,english
0,"new jersey is sometimes quiet during autumn , ..."
1,the united states is usually chilly during jul...
2,"california is usually quiet during march , and..."
3,the united states is sometimes mild during jun...
4,"your least liked fruit is the grape , but my l..."
...,...
137855,"france is never busy during march , and it is ..."
137856,"india is sometimes beautiful during spring , a..."
137857,"india is never wet during summer , but it is s..."
137858,"france is never chilly during january , but it..."


In [ ]:
df_french

In [ ]:
df=pd.concat([df_english,df_french],axis=1)

In [ ]:
df

,0
0,the united states is sometimes dry during may ...


Function To remove punctuations

In [ ]:
def remove_punc(x):
  return re.sub('[!#?,.:";]','',x)

Removing punctuations

In [ ]:
df['french']=df['french'].apply(remove_punc)
df['english']=df['english'].apply(remove_punc)

In [ ]:
english_words=[]
french_words=[]

Function to find unique words

In [ ]:
def get_unique_words(x,word_list):
  for word in x.split():
    if word not in word_list:
      word_list.append(word)


Collecting The Unique words in the whole set

In [ ]:
df['english'].apply(lambda x:get_unique_words(x,english_words))
df['french'].apply(lambda x:get_unique_words(x,french_words))
total_english_words=len(english_words)
print(total_english_words)
total_french_words=len(french_words)

199


In [ ]:
def get_all_words(x,word_list):
  for word in x.split():
      word_list.append(word)

In [ ]:
all_english_words=[]

In [ ]:
df['english'].apply(lambda x:get_all_words(x,all_english_words))

english_count=Counter(all_english_words)
sorted(english_count.items(),reverse=True,key=operator.itemgetter(1))

In [ ]:
fig=px.bar(x=english_words,y=english_count)
fig.show()

In [ ]:
max_len_english=-1
max_len_french=-1
def max_length(max_len,name):
  for doc in df[name]:
    tokens=nltk.word_tokenize(doc)
    if max_len<len(tokens):
      max_len=len(tokens)
  return max_len



max_len_english=max_length(max_len_english,'english')
max_len_french=max_length(max_len_french,'french')

print(max_len_french)


23


In [ ]:
def tokenize_and_pad(x, maxlen):
  #  a tokenier to tokenize the words and create sequences of tokenized words
  tokenizer = Tokenizer(char_level = False)
  tokenizer.fit_on_texts(x)
  sequences = tokenizer.texts_to_sequences(x)
  padded = pad_sequences(sequences, maxlen = maxlen, padding = 'post')
  return tokenizer, sequences, padded

In [ ]:
df['english'].shape

(137860,)

In [ ]:
x_tokenizer, x_sequences, x_padded = tokenize_and_pad(df['english'], max_len_english)
y_tokenizer, y_sequences, y_padded = tokenize_and_pad(df.french, max_len_french)

In [ ]:
x_padded.shape

(137860, 15)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_padded,y_padded,test_size=0.1)

In [ ]:
x_train.shape

(124074, 15)

In [ ]:
print(len(x_padded[0]))
print(len(y_padded[0]))

19
19


In [ ]:
total_english_vocabs_size = total_english_words + 1
total_french_vocabs_size = total_french_words + 1

model=Sequential()

model.add(Embedding(total_english_vocabs_size,256,input_length=max_len_english,mask_zero=True))

model.add(LSTM(256))

model.add(RepeatVector(max_len_french))

model.add(LSTM(256,return_sequences=True))

model.add(TimeDistributed(Dense(total_french_vocabs_size,activation='softmax')))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'],run_eagerly=True)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 256)           51200     
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 repeat_vector_1 (RepeatVec  (None, 23, 256)           0         
 tor)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 23, 256)           525312    
                                                                 
 time_distributed_1 (TimeDi  (None, 23, 351)           90207     
 stributed)                                                      
                                                                 
Total params: 1192031 (4.55 MB)
Trainable params: 1192

In [ ]:
y_train=np.expand_dims(y_train,axis=2)
y_train.shape

(124074, 23, 1)

In [ ]:
x_train.shape

(124074, 15)

In [ ]:
model.fit(x_train,y_train,batch_size=1024,validation_split=0.1,epochs=3)

Epoch 1/20
110/110 [==============================] - 442s 4s/step - loss: 2.6975 - accuracy: 0.4976 - val_loss: 2.0945 - val_accuracy: 0.5458
Epoch 2/20
110/110 [==============================] - 433s 4s/step - loss: 1.8649 - accuracy: 0.5715 - val_loss: 1.6247 - val_accuracy: 0.5957
Epoch 3/20
110/110 [==============================] - 431s 4s/step - loss: 1.5110 - accuracy: 0.6137 - val_loss: 1.3992 - val_accuracy: 0.6331
Epoch 4/20
110/110 [==============================] - 430s 4s/step - loss: 1.3129 - accuracy: 0.6513 - val_loss: 1.2287 - val_accuracy: 0.6655
Epoch 5/20
110/110 [==============================] - 425s 4s/step - loss: 1.1457 - accuracy: 0.6785 - val_loss: 1.2107 - val_accuracy: 0.6585
Epoch 6/20
110/110 [==============================] - 434s 4s/step - loss: 1.0114 - accuracy: 0.7107 - val_loss: 0.9495 - val_accuracy: 0.7268
Epoch 7/20
110/110 [==============================] - 442s 4s/step - loss: 0.8861 - accuracy: 0.7455 - val_loss: 0.8817 - val_accuracy: 0.7355

In [ ]:
model.save('weights.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
def prediction(x, x_tokenizer = x_tokenizer, y_tokenizer = y_tokenizer):
    predictions = model.predict(x)[0]
    id_to_word = {id: word for word, id in y_tokenizer.word_index.items()}
    id_to_word[0] = ''
    return ' '.join([id_to_word[j] for j in np.argmax(predictions,1)])

In [ ]:
def pad_to_text(padded, tokenizer):

    id_to_word = {id: word for word, id in tokenizer.word_index.items()}
    id_to_word[0] = ''

    return ' '.join([id_to_word[j] for j in padded])

In [ ]:
for i in range(5):

  print('Original English word - {}\n'.format(pad_to_text(x_test[i], x_tokenizer)))
  print('Original French word - {}\n'.format(pad_to_text(y_test[i], y_tokenizer)))
  print('Predicted French word - {}\n\n\n\n'.format(prediction(x_test[i:i+1])+f'   {x_test[i:i+1]}'))

Original English word - the united states is sometimes dry during may but it is chilly in june 

Original French word - les états unis est parfois sec au mois de mai mais il est froid en juin       

1/1 [==============================] - 0s 202ms/step
Predicted French word - la états est est est jamais au en mais mais il est est en en           [[ 5 20 21  1  8 59  4 40  6  3  1 62  2 34  0]]




Original English word - california is sometimes freezing during november and it is cold in summer   

Original French word - la californie est parfois le gel en novembre et il fait froid en été         

1/1 [==============================] - 0s 167ms/step
Predicted French word - paris est est parfois en en et et il est est est en en            [[22  1  8 51  4 45  7  3  1 57  2 46  0  0  0]]




Original English word - india is wonderful during july and it is usually snowy in august   

Original French word - l' inde est merveilleux en juillet et il est généralement enneigée en août         

In [ ]:
x_test[1]

array([22,  1,  8, 51,  4, 45,  7,  3,  1, 57,  2, 46,  0,  0,  0],
      dtype=int32)

In [ ]:
data={'the united states is sometimes dry during may but it is chilly in june'}
df = pd.DataFrame(data)

In [ ]:
word=['california is sometimes freezing during november and it is cold in summer ']
ia,b,input_sentence=tokenize_and_pad(word,max_len_english)


print(' French Translation - {}\n'.format(prediction(input_sentence)))
print(input_sentence)

1/1 [==============================] - 0s 110ms/step
 French Translation - la est est est en en mais mais il est est en en          

[[ 2  1  3  4  5  6  7  8  1  9 10 11  0  0  0]]
